In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [32]:
en = pd.read_csv("/content/drive/MyDrive/siqi mock/csv files/final_all_lyrics_negpos.csv", encoding='utf-8')

In [33]:
en

,artist,song,lyrics,bert_nega,bert_posi,bert_score,nltk_neg,nltk_neu,nltk_pos,nltk_score
0,Miley Cyrus,Flowers,"We were good, we were gold Kind of dream that ...",13,9,-0.144740,0,15,7,0.238127
1,SZA,Kill Bill,I'm still a fan even though I was salty Hate t...,15,7,-0.207048,2,16,4,0.054209
2,"Metro Boomin, The Weeknd, 21 Savage",Creepin' (with The Weeknd & 21 Savage),Oooh Just can't believe this man (Metro Boomin...,17,7,-0.239239,0,24,0,0.047654
3,"Metro Boomin, Future, Chris Brown",Superhero (Heroes & Villains) [with Future & C...,"(Metro, metro, metro, metro) Drankin' dope tur...",28,5,-0.427365,1,30,2,-0.013979
4,"Drake, 21 Savage",Rich Flex,"Go buy a zip of weed, hit the club Pay for 'bo...",18,10,-0.190593,1,25,2,-0.034607
...,...,...,...,...,...,...,...,...,...,...
241,"Grupo Marca Registrada, Grupo Frontera",Di Que Si,He was a boy who was a dreamer And he flew so ...,10,2,-0.383420,0,12,0,-0.024825
242,Mae Stephens,If We Ever Broke Up,"Throw it, throw it up Watch it all fall out Po...",17,16,-0.042716,0,33,0,0.002339
243,"$uicideboy$, Shakewell",Big Shot Cream Soda,"Rain, he wanted it comfortable I wanted that p...",10,10,-0.011087,2,13,5,0.135535
244,Chino Pacas,El Gordo Trae El Mando,P-P-P-P-P-PARTY That smile on your face Makes ...,22,4,-0.432062,1,19,6,0.001535


#hit / nohit (100)

In [34]:
dataset = pd.read_csv("/content/drive/MyDrive/siqi mock/csv files/final_all_data.csv", encoding = 'utf-8')

In [35]:
rank_list = dataset["rank"].values.tolist()
top100 = []

for rank in rank_list:
    if rank < 101:
        top100.append(1)
    else:
        top100.append(0)

dataset["top100"] = top100
dataset

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,danceability,...,duration_ms,time_signature,bert_nega,bert_posi,bert_score,nltk_neg,nltk_neu,nltk_pos,nltk_score,top100
0,1,spotify:track:0yLdNVWF3Srea0uzk55zFn,Miley Cyrus,Flowers,Columbia,1,1,3,18441075,0.707,...,200455,4,13.0,9.0,-0.144740,0.0,15.0,7.0,0.238127,1
1,2,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,SZA,Kill Bill,Top Dawg Entertainment/RCA Records,1,2,8,15313761,0.644,...,153947,4,15.0,7.0,-0.207048,2.0,16.0,4.0,0.054209,1
2,3,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,"Metro Boomin, The Weeknd, 21 Savage",Creepin' (with The Weeknd & 21 Savage),Republic Records,1,3,9,8185172,0.715,...,221520,4,17.0,7.0,-0.239239,0.0,24.0,0.0,0.047654,1
3,4,spotify:track:0vjeOZ3Ft5jvAi9SBFJm1j,"Metro Boomin, Future, Chris Brown",Superhero (Heroes & Villains) [with Future & C...,Republic Records,2,4,9,7350116,0.526,...,182667,4,28.0,5.0,-0.427365,1.0,30.0,2.0,-0.013979,1
4,5,spotify:track:1bDbXMyjaUIooNwFE9wn0N,"Drake, 21 Savage",Rich Flex,OVO / Republic Records,1,5,13,7109310,0.561,...,239360,3,18.0,10.0,-0.190593,1.0,25.0,2.0,-0.034607,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,195,spotify:track:4S4QJfBGGrC8jRIjJHf1Ka,PinkPantheress,Pain,Parlophone UK,48,183,20,1924113,0.829,...,98544,4,8.0,11.0,0.091898,1.0,18.0,0.0,0.038616,0
785,196,spotify:track:01Lr5YepbgjXAWR9iOEyH1,Chief Keef,Love Sosa,UMGRI Interscope,44,178,55,1921806,0.772,...,246213,4,18.0,2.0,-0.459613,4.0,12.0,4.0,-0.122535,0
786,197,spotify:track:5OELUCYgOHKFAvCERnAvfS,Morgan Wallen,More Than My Hometown,Big Loud / Republic,21,-1,99,1902147,0.620,...,216573,4,29.0,13.0,-0.236389,1.0,39.0,2.0,0.047281,0
787,198,spotify:track:58k32my5lKofeZRtIvBDg9,Baby Keem,HONEST,Baby Keem LLC,39,196,29,1900952,0.676,...,172672,4,13.0,11.0,-0.056934,1.0,22.0,1.0,0.028050,0


In [36]:
train_data = dataset.drop(['uri', 'rank', 'top100'], axis=1)
test_data = dataset[['top100']]

train_dummy = pd.get_dummies(train_data)
test_dummy = pd.get_dummies(test_data)

X_train, X_test, y_train, y_test = train_test_split(train_dummy, test_dummy)

In [37]:
X_train

,peak_rank,previous_rank,weeks_on_chart,streams,danceability,energy,key,loudness,mode,speechiness,...,analysis_url_https://api.spotify.com/v1/audio-analysis/7KXjTSCq5nL1LoYtL7XAwS,analysis_url_https://api.spotify.com/v1/audio-analysis/7KokYm8cMIXCsGVmUvKtqf,analysis_url_https://api.spotify.com/v1/audio-analysis/7LR85XLWw2yXqKBSI5brbG,analysis_url_https://api.spotify.com/v1/audio-analysis/7MXVkk9YMctZqd1Srtv4MB,analysis_url_https://api.spotify.com/v1/audio-analysis/7fBv7CLKzipRk6EC6TWHOB,analysis_url_https://api.spotify.com/v1/audio-analysis/7jtQIBanIiJOMS6RyCx6jZ,analysis_url_https://api.spotify.com/v1/audio-analysis/7oLKoGzQVwjZ91AzCn560T,analysis_url_https://api.spotify.com/v1/audio-analysis/7sO5G9EABYOXQKNPNiE9NR,analysis_url_https://api.spotify.com/v1/audio-analysis/7ySUcLPVX7KudhnmNcgY2D,analysis_url_https://api.spotify.com/v1/audio-analysis/7zy7iE31dGO70ShMXghsfj
627,38,66,5,3581569,0.768,0.740,0,-6.595,1,0.3360,...,0,0,0,0,0,0,0,0,0,0
617,5,26,6,3986721,0.778,0.632,2,-5.600,0,0.0493,...,0,0,0,0,0,0,0,0,0,0
382,2,167,26,1934122,0.951,0.878,2,-2.565,1,0.2110,...,0,0,0,0,0,0,0,0,0,0
322,65,150,19,2265633,0.583,0.344,2,-8.765,1,0.0559,...,0,0,0,0,0,0,0,0,0,0
512,27,106,104,2374875,0.585,0.564,0,-7.063,0,0.0515,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,7,193,61,1912291,0.837,0.636,1,-7.643,1,0.0860,...,0,0,0,0,0,0,0,0,0,0
1,1,2,8,15313761,0.644,0.735,8,-5.747,1,0.0391,...,0,0,0,0,0,0,0,0,0,0
623,2,35,36,3740921,0.529,0.673,0,-4.711,1,0.1750,...,0,0,0,0,0,0,0,0,0,0
302,43,126,194,2386800,0.614,0.680,6,-4.580,1,0.0289,...,0,0,0,0,0,0,0,0,0,0


In [38]:
#X_train.to_csv("0110_top100_X_train.csv", encoding='utf-8', index=False)
#X_test.to_csv("0110_top100_X_test.csv", encoding='utf-8', index=False)
#y_train.to_csv("0110_top100_y_train.csv", encoding='utf-8', index=False)
#y_test.to_csv("0110_top100_y_test.csv", encoding='utf-8', index=False)

In [39]:
#X_train_dummy.to_csv("0110_top100_X_train_dummy.csv", encoding='utf-8', index=False)
#X_test_dummy.to_csv("0110_top100_X_test_dummy.csv", encoding='utf-8', index=False)
#y_train_dummy.to_csv("0110_top100_y_train_dummy.csv", encoding='utf-8', index=False)
#y_test_dummy.to_csv("0110_top100_y_test_dummy.csv", encoding='utf-8', index=False)

##New Dataset

##RF

In [40]:
forest = RandomForestClassifier(n_estimators=130, max_features=2, n_jobs=-1, random_state=0)
forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

train_accu_rf = forest.score(X_train, y_train)
print("Train model accuracy:{}".format(train_accu_rf))

test_acc_rf = accuracy_score(y_test, y_pred)
print("Test accuracy:{}".format(test_acc_rf))

confusion = confusion_matrix(y_test, y_pred)
print(confusion)

<ipython-input-40-8ed3c29bdbeb>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X_train, y_train)


Train model accuracy:1.0
Test accuracy:0.9292929292929293
[[87  6]
 [ 8 97]]


In [41]:
tn = 80
fp = 8
fn = 8
tp = 102

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

Accu: 0.9191919191919192
Prec: 0.9272727272727272
Recall: 0.9272727272727272
F-measure: 0.9272727272727274


#GB

In [42]:
# GB

gb = GradientBoostingClassifier(learning_rate=0.05, n_estimators=60, max_depth=3)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
confusion_gb = confusion_matrix(y_test, y_pred_gb)
print(confusion_gb)

train_accu_gb = gb.score(X_train, y_train)
print("Train model accuracy:{}".format(train_accu_gb))

test_acc_gb = accuracy_score(y_test, y_pred_gb)
print("Test accuracy:{}".format(test_acc_gb))

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 92   1]
 [  3 102]]
Train model accuracy:0.9932318104906938
Test accuracy:0.9797979797979798


In [43]:
tn = 86
fp = 2
fn = 6
tp = 104

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

Accu: 0.9595959595959596
Prec: 0.9811320754716981
Recall: 0.9454545454545454
F-measure: 0.9797101449275362


#SVM

In [44]:
# SVM with MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc = SVC(kernel="rbf", C=10, gamma=0.1)
svc.fit(X_train_scaled, y_train)

y_pred_svc = svc.predict(X_test_scaled)
confusion_svc = confusion_matrix(y_test, y_pred_svc)
print(confusion_svc)

train_accu_svc = svc.score(X_train_scaled, y_train)
print("Train model accuracy:{}".format(train_accu_svc))

test_acc_svc = accuracy_score(y_test, y_pred_svc)
print("Train model accuracy:{}".format(test_acc_svc))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[84  9]
 [11 94]]
Train model accuracy:0.9593908629441624
Train model accuracy:0.898989898989899


In [45]:
tn = 74
fp = 14
fn = 7
tp = 103

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

Accu: 0.8939393939393939
Prec: 0.8803418803418803
Recall: 0.9363636363636364
F-measure: 0.8823723228995057


#Default Dataset

In [47]:
X_train_de = X_train.drop(['bert_nega','bert_posi','bert_score','nltk_neg',
                        'nltk_neu','nltk_pos', 'nltk_score'], axis=1)

X_test_de = X_test.drop(['bert_nega','bert_posi','bert_score','nltk_neg',
                        'nltk_neu','nltk_pos', 'nltk_score'], axis=1)

#RF

In [48]:
# RF

forest = RandomForestClassifier(n_estimators=50, max_features=1, n_jobs=-1, random_state=40)
forest.fit(X_train_de, y_train)

y_pred = forest.predict(X_test_de)

train_accu_rf = forest.score(X_train_de, y_train)
print("Train model accuracy:{}".format(train_accu_rf))

test_acc_rf = accuracy_score(y_test, y_pred)
print("Test accuracy:{}".format(test_acc_rf))

confusion = confusion_matrix(y_test, y_pred)
print(confusion)

<ipython-input-48-848bb143761d>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X_train_de, y_train)


Train model accuracy:1.0
Test accuracy:0.9292929292929293
[[87  6]
 [ 8 97]]


In [49]:
tn = 79
fp = 9
fn = 7
tp = 103

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

Accu: 0.9191919191919192
Prec: 0.9196428571428571
Recall: 0.9363636363636364
F-measure: 0.920274914089347


#GB

In [50]:
gb = GradientBoostingClassifier(learning_rate=0.05, n_estimators=60, max_depth=2)
gb.fit(X_train_de, y_train)
y_pred_gb = gb.predict(X_test_de)
confusion_gb = confusion_matrix(y_test, y_pred_gb)
print(confusion_gb)

train_accu_gb = gb.score(X_train_de, y_train)
print("Train model accuracy:{}".format(train_accu_gb))

test_acc_gb = accuracy_score(y_test, y_pred_gb)
print("Test accuracy:{}".format(test_acc_gb))

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 92   1]
 [  3 102]]
Train model accuracy:0.9847715736040609
Test accuracy:0.9797979797979798


In [51]:
tn = 92
fp = 1
fn = 3
tp = 102

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

Accu: 0.9797979797979798
Prec: 0.9902912621359223
Recall: 0.9714285714285714
F-measure: 0.9895522388059702


#SVM

In [52]:
scaler = MinMaxScaler()
scaler.fit(X_train_de)
X_train_scaled = scaler.transform(X_train_de)
X_test_scaled = scaler.transform(X_test_de)

svc = SVC(kernel="rbf", C=10, gamma=0.01)
svc.fit(X_train_scaled, y_train)

y_pred_svc = svc.predict(X_test_scaled)
confusion_svc = confusion_matrix(y_test, y_pred_svc)
print(confusion_svc)

train_accu_svc = svc.score(X_train_scaled, y_train)
print("Train model accuracy:{}".format(train_accu_svc))

test_acc_svc = accuracy_score(y_test, y_pred_svc)
print("Train model accuracy:{}".format(test_acc_svc))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[82 11]
 [10 95]]
Train model accuracy:0.9543147208121827
Train model accuracy:0.8939393939393939


In [53]:
tn = 82
fp = 11
fn = 10
tp = 95

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

Accu: 0.8939393939393939
Prec: 0.8962264150943396
Recall: 0.9047619047619048
F-measure: 0.8965517241379312


#50/50

In [54]:
rank_list = dataset["rank"].values.tolist()
top50 = []

for rank in rank_list:
    if rank < 51:
        top50.append(1)
    else:
        top50.append(0)

dataset["top50"] = top50

In [55]:
top50 = dataset.query("top50 == 1")
top50

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,danceability,...,time_signature,bert_nega,bert_posi,bert_score,nltk_neg,nltk_neu,nltk_pos,nltk_score,top100,top50
0,1,spotify:track:0yLdNVWF3Srea0uzk55zFn,Miley Cyrus,Flowers,Columbia,1,1,3,18441075,0.707,...,4,13.0,9.0,-0.144740,0.0,15.0,7.0,0.238127,1,1
1,2,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,SZA,Kill Bill,Top Dawg Entertainment/RCA Records,1,2,8,15313761,0.644,...,4,15.0,7.0,-0.207048,2.0,16.0,4.0,0.054209,1,1
2,3,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,"Metro Boomin, The Weeknd, 21 Savage",Creepin' (with The Weeknd & 21 Savage),Republic Records,1,3,9,8185172,0.715,...,4,17.0,7.0,-0.239239,0.0,24.0,0.0,0.047654,1,1
3,4,spotify:track:0vjeOZ3Ft5jvAi9SBFJm1j,"Metro Boomin, Future, Chris Brown",Superhero (Heroes & Villains) [with Future & C...,Republic Records,2,4,9,7350116,0.526,...,4,28.0,5.0,-0.427365,1.0,30.0,2.0,-0.013979,1,1
4,5,spotify:track:1bDbXMyjaUIooNwFE9wn0N,"Drake, 21 Savage",Rich Flex,OVO / Republic Records,1,5,13,7109310,0.561,...,3,18.0,10.0,-0.190593,1.0,25.0,2.0,-0.034607,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,46,spotify:track:4sx6NRwL6Ol3V6m9exwGlQ,The Kid LAROI,Love Again,Columbia,13,29,4,3251724,0.662,...,4,8.0,5.0,-0.150961,0.0,11.0,2.0,0.045092,1,1
636,47,spotify:track:0WtM2NBVQNNJLh6scP13H8,"Rema, Selena Gomez",Calm Down (with Selena Gomez),Mavin Records / Jonzing World,47,63,23,3223919,0.801,...,4,21.0,3.0,-0.452054,4.0,20.0,0.0,0.092417,1,1
637,48,spotify:track:7EAMXbLcL0qXmciM5SwMh2,Mac DeMarco,Heart To Heart,Mac’s Record Label,30,51,10,3214516,0.901,...,4,16.0,10.0,-0.132072,0.0,24.0,2.0,0.072327,1,1
638,49,spotify:track:5BbdKBZO0TH0GhfxUfyhL9,Lil Baby,Freestyle,Quality Control Music (Caroline),49,74,36,3202823,0.877,...,4,13.0,8.0,-0.139174,2.0,15.0,4.0,0.090119,1,1


In [56]:
random = dataset.query("top50 == 0")

In [58]:
list_random = random.sample(n=200)
list_random

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,danceability,...,time_signature,bert_nega,bert_posi,bert_score,nltk_neg,nltk_neu,nltk_pos,nltk_score,top100,top50
456,65,spotify:track:7LR85XLWw2yXqKBSI5brbG,"JAY-Z, Rihanna, Kanye West",Run This Town,Roc Nation / Jay-Z,65,-1,1,3160742,0.632,...,4,25.0,7.0,-0.332954,0.0,24.0,8.0,0.135100,1,0
480,89,spotify:track:1uN0f4NqMDcReSwXO6PLuG,Rihanna,Where Have You Been,Def Jam Recordings,89,-1,1,2721856,0.719,...,4,42.0,13.0,-0.308182,1.0,40.0,14.0,-0.042425,1,0
547,156,spotify:track:2ZL7WZcjuYKi1KUDtp4kCC,"Drake, 21 Savage",Spin Bout U,OVO / Republic Records,6,134,15,2134958,0.773,...,4,22.0,13.0,-0.146474,1.0,31.0,3.0,0.118557,0,0
490,99,spotify:track:5UWwZ5lm5PKu6eKsHAGxOk,Foo Fighters,Everlong,RCA Records Label,60,82,28,2637894,0.413,...,4,17.0,12.0,-0.112638,1.0,24.0,4.0,-0.012379,1,0
256,62,spotify:track:1qEmFfgcLObUfQm0j1W2CK,Harry Styles,Late Night Talking,Columbia,2,84,38,3132054,0.714,...,4,13.0,5.0,-0.280974,0.0,16.0,2.0,0.074806,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,79,spotify:track:4tqcoej1zPvwePZCzuAjJd,J. Cole,Wet Dreamz,Roc Nation Records LLC,69,85,71,2670469,0.495,...,4,28.0,4.0,-0.443876,1.0,27.0,4.0,0.071431,1,0
475,84,spotify:track:7MXVkk9YMctZqd1Srtv4MB,"The Weeknd, Daft Punk",Starboy,Universal Republic Records,3,83,107,2776869,0.679,...,4,16.0,15.0,-0.016620,1.0,30.0,0.0,0.015716,1,0
541,150,spotify:track:6Ox1rvlwEpB49drasQm6RF,Lil Peep,Star Shopping,Lil Peep,100,127,67,2169464,0.538,...,4,13.0,5.0,-0.254568,1.0,15.0,2.0,0.192900,0,0
308,114,spotify:track:577YBGuskWkVDCxZrLRB4v,Lil Baby,Pure Cocaine,Quality Control Music/Motown Records/Capitol R...,52,121,42,2348366,0.964,...,4,27.0,6.0,-0.381267,2.0,29.0,2.0,-0.017415,0,0


In [59]:
dataset_1 = pd.concat([top50, list_random])
dataset_1

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,danceability,...,time_signature,bert_nega,bert_posi,bert_score,nltk_neg,nltk_neu,nltk_pos,nltk_score,top100,top50
0,1,spotify:track:0yLdNVWF3Srea0uzk55zFn,Miley Cyrus,Flowers,Columbia,1,1,3,18441075,0.707,...,4,13.0,9.0,-0.144740,0.0,15.0,7.0,0.238127,1,1
1,2,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,SZA,Kill Bill,Top Dawg Entertainment/RCA Records,1,2,8,15313761,0.644,...,4,15.0,7.0,-0.207048,2.0,16.0,4.0,0.054209,1,1
2,3,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,"Metro Boomin, The Weeknd, 21 Savage",Creepin' (with The Weeknd & 21 Savage),Republic Records,1,3,9,8185172,0.715,...,4,17.0,7.0,-0.239239,0.0,24.0,0.0,0.047654,1,1
3,4,spotify:track:0vjeOZ3Ft5jvAi9SBFJm1j,"Metro Boomin, Future, Chris Brown",Superhero (Heroes & Villains) [with Future & C...,Republic Records,2,4,9,7350116,0.526,...,4,28.0,5.0,-0.427365,1.0,30.0,2.0,-0.013979,1,1
4,5,spotify:track:1bDbXMyjaUIooNwFE9wn0N,"Drake, 21 Savage",Rich Flex,OVO / Republic Records,1,5,13,7109310,0.561,...,3,18.0,10.0,-0.190593,1.0,25.0,2.0,-0.034607,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,79,spotify:track:4tqcoej1zPvwePZCzuAjJd,J. Cole,Wet Dreamz,Roc Nation Records LLC,69,85,71,2670469,0.495,...,4,28.0,4.0,-0.443876,1.0,27.0,4.0,0.071431,1,0
475,84,spotify:track:7MXVkk9YMctZqd1Srtv4MB,"The Weeknd, Daft Punk",Starboy,Universal Republic Records,3,83,107,2776869,0.679,...,4,16.0,15.0,-0.016620,1.0,30.0,0.0,0.015716,1,0
541,150,spotify:track:6Ox1rvlwEpB49drasQm6RF,Lil Peep,Star Shopping,Lil Peep,100,127,67,2169464,0.538,...,4,13.0,5.0,-0.254568,1.0,15.0,2.0,0.192900,0,0
308,114,spotify:track:577YBGuskWkVDCxZrLRB4v,Lil Baby,Pure Cocaine,Quality Control Music/Motown Records/Capitol R...,52,121,42,2348366,0.964,...,4,27.0,6.0,-0.381267,2.0,29.0,2.0,-0.017415,0,0


In [60]:
train_data = dataset.drop(['uri', 'rank', 'top100'], axis=1)
test_data = dataset[['top100']]

train_dummy = pd.get_dummies(train_data)
test_dummy = pd.get_dummies(test_data)

X_train, X_test, y_train, y_test = train_test_split(train_dummy, test_dummy)

#NEW

In [72]:
forest = RandomForestClassifier(n_estimators=50, max_features=2, n_jobs=-1, random_state=0)
forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

train_accu_rf = forest.score(X_train, y_train)
print("Train model accuracy:{}".format(train_accu_rf))

test_acc_rf = accuracy_score(y_test, y_pred)
print("Test accuracy:{}".format(test_acc_rf))

confusion = confusion_matrix(y_test, y_pred)
print(confusion)

<ipython-input-72-8001dd1ee3c2>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X_train, y_train)


Train model accuracy:1.0
Test accuracy:0.898989898989899
[[90 12]
 [ 8 88]]


In [62]:
# GB

gb = GradientBoostingClassifier(learning_rate=0.05, n_estimators=60, max_depth=3)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
confusion_gb = confusion_matrix(y_test, y_pred_gb)
print(confusion_gb)

train_accu_gb = gb.score(X_train, y_train)
print("Train model accuracy:{}".format(train_accu_gb))

test_acc_gb = accuracy_score(y_test, y_pred_gb)
print("Test accuracy:{}".format(test_acc_gb))

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[99  3]
 [ 3 93]]
Train model accuracy:0.9983079526226735
Test accuracy:0.9696969696969697


In [63]:
# SVM with MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc = SVC(kernel="rbf", C=10, gamma=0.1)
svc.fit(X_train_scaled, y_train)

y_pred_svc = svc.predict(X_test_scaled)
confusion_svc = confusion_matrix(y_test, y_pred_svc)
print(confusion_svc)

train_accu_svc = svc.score(X_train_scaled, y_train)
print("Train model accuracy:{}".format(train_accu_svc))

test_acc_svc = accuracy_score(y_test, y_pred_svc)
print("Train model accuracy:{}".format(test_acc_svc))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[88 14]
 [15 81]]
Train model accuracy:0.9627749576988156
Train model accuracy:0.8535353535353535


#DEFAULT

In [64]:
X_train_de = X_train.drop(['bert_nega','bert_posi','bert_score','nltk_neg',
                        'nltk_neu','nltk_pos', 'nltk_score'], axis=1)

X_test_de = X_test.drop(['bert_nega','bert_posi','bert_score','nltk_neg',
                        'nltk_neu','nltk_pos', 'nltk_score'], axis=1)

In [65]:
# RF

forest = RandomForestClassifier(n_estimators=130, max_features=2, n_jobs=-1, random_state=0)
forest.fit(X_train_de, y_train)

y_pred = forest.predict(X_test_de)

train_accu_rf = forest.score(X_train_de, y_train)
print("Train model accuracy:{}".format(train_accu_rf))

test_acc_rf = accuracy_score(y_test, y_pred)
print("Test accuracy:{}".format(test_acc_rf))

confusion = confusion_matrix(y_test, y_pred)
print(confusion)

<ipython-input-65-98e038efdd00>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X_train_de, y_train)


Train model accuracy:1.0
Test accuracy:0.8787878787878788
[[88 14]
 [10 86]]


In [74]:
# GB

gb = GradientBoostingClassifier(learning_rate=0.05, n_estimators=60, max_depth=3)
gb.fit(X_train_de, y_train)
y_pred_gb = gb.predict(X_test_de)
confusion_gb = confusion_matrix(y_test, y_pred_gb)
print(confusion_gb)

train_accu_gb = gb.score(X_train_de, y_train)
print("Train model accuracy:{}".format(train_accu_gb))

test_acc_gb = accuracy_score(y_test, y_pred_gb)
print("Test accuracy:{}".format(test_acc_gb))

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[99  3]
 [ 3 93]]
Train model accuracy:1.0
Test accuracy:0.9696969696969697


In [67]:
# SVM with MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train_de)
X_train_scaled = scaler.transform(X_train_de)
X_test_scaled = scaler.transform(X_test_de)

svc = SVC(kernel="rbf", C=10, gamma=0.01)
svc.fit(X_train_scaled, y_train)

y_pred_svc = svc.predict(X_test_scaled)
confusion_svc = confusion_matrix(y_test, y_pred_svc)
print(confusion_svc)

train_accu_svc = svc.score(X_train_scaled, y_train)
print("Train model accuracy:{}".format(train_accu_svc))

test_acc_svc = accuracy_score(y_test, y_pred_svc)
print("Train model accuracy:{}".format(test_acc_svc))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[89 13]
 [13 83]]
Train model accuracy:0.9627749576988156
Train model accuracy:0.8686868686868687


In [68]:
# RF

tn = 89
fp = 13
fn = 10
tp = 86

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------New---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))


tn = 88
fp = 14
fn = 10
tp = 86

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------De---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

-----------------De---------------------
Accu: 0.8838383838383839
Prec: 0.8686868686868687
Recall: 0.8958333333333334
F-measure: 0.8697005056398289
-----------------New---------------------
Accu: 0.8787878787878788
Prec: 0.86
Recall: 0.8958333333333334
F-measure: 0.8613251155624037


In [75]:
# GB

tn = 99
fp = 3
fn = 3
tp = 93

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------New---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))


tn = 99
fp = 3
fn = 3
tp = 93

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------De---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

-----------------New---------------------
Accu: 0.9696969696969697
Prec: 0.96875
Recall: 0.96875
F-measure: 0.96875
-----------------De---------------------
Accu: 0.9696969696969697
Prec: 0.96875
Recall: 0.96875
F-measure: 0.96875


In [76]:
# SVM

tn = 88
fp = 14
fn = 15
tp = 81

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------New---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))


tn = 89
fp = 13
fn = 13
tp = 87

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------De---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

-----------------New---------------------
Accu: 0.8535353535353535
Prec: 0.8526315789473684
Recall: 0.84375
F-measure: 0.8522865236746257
-----------------De---------------------
Accu: 0.8712871287128713
Prec: 0.87
Recall: 0.87
F-measure: 0.8699999999999999


#50 vs 150

In [77]:
rank_list = dataset["rank"].values.tolist()
top50 = []

for rank in rank_list:
    if rank < 51:
        top50.append(1)
    else:
        top50.append(0)

dataset["top50"] = top50

In [79]:
train_data = dataset.drop(['uri', 'rank', 'top50'], axis=1)
test_data = dataset[['top50']]

#Convert categorical variable into dummy variables
train_dummy = pd.get_dummies(train_data)
test_dummy = pd.get_dummies(test_data)

X_train, X_test, y_train, y_test = train_test_split(train_dummy, test_dummy)

In [80]:
y_test.reset_index(inplace=True, drop=True)

In [81]:
forest = RandomForestClassifier(n_estimators=50, max_features=2, n_jobs=-1, random_state=0)
forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

train_accu_rf = forest.score(X_train, y_train)
print("Train model accuracy:{}".format(train_accu_rf))

test_acc_rf = accuracy_score(y_test, y_pred)
print("Test accuracy:{}".format(test_acc_rf))

confusion = confusion_matrix(y_test, y_pred)
print(confusion)

<ipython-input-81-8001dd1ee3c2>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X_train, y_train)


Train model accuracy:1.0
Test accuracy:0.9444444444444444
[[145   5]
 [  6  42]]


In [83]:
y_pred_rf_new = y_pred
df = pd.DataFrame(y_pred_rf_new, columns=["rf"])
df

,rf
0,0
1,1
2,0
3,0
4,0
...,...
193,0
194,0
195,0
196,0


In [84]:
# GB

gb = GradientBoostingClassifier(learning_rate=0.05, n_estimators=60, max_depth=3)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
confusion_gb = confusion_matrix(y_test, y_pred_gb)
print(confusion_gb)

train_accu_gb = gb.score(X_train, y_train)
print("Train model accuracy:{}".format(train_accu_gb))

test_acc_gb = accuracy_score(y_test, y_pred_gb)
print("Test accuracy:{}".format(test_acc_gb))

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[148   2]
 [  3  45]]
Train model accuracy:1.0
Test accuracy:0.9747474747474747


In [85]:
y_pred_gb_new = y_pred_gb
df1 = pd.DataFrame(data=y_pred_gb_new, columns=["gb"])
df1

,gb
0,0
1,1
2,0
3,0
4,0
...,...
193,0
194,0
195,0
196,0


In [86]:
# SVM with MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc = SVC(kernel="rbf", C=10, gamma=0.1)
svc.fit(X_train_scaled, y_train)

y_pred_svc = svc.predict(X_test_scaled)
confusion_svc = confusion_matrix(y_test, y_pred_svc)
print(confusion_svc)

train_accu_svc = svc.score(X_train_scaled, y_train)
print("Train model accuracy:{}".format(train_accu_svc))

test_acc_svc = accuracy_score(y_test, y_pred_svc)
print("Train model accuracy:{}".format(test_acc_svc))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[145   5]
 [ 10  38]]
Train model accuracy:0.9661590524534687
Train model accuracy:0.9242424242424242


In [87]:
y_pred_svc_new = y_pred_svc
df2 = pd.DataFrame(y_pred_svc_new, columns=["svm"])
df_new = pd.concat([df, df1, df2], axis=1)
df_new

,rf,gb,svm
0,0,0,0
1,1,1,1
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
193,0,0,0
194,0,0,0
195,0,0,0
196,0,0,0


In [88]:
df_new = pd.concat([df_new, y_test], axis=1)
df_new

,rf,gb,svm,top50
0,0,0,0,0
1,1,1,1,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
193,0,0,0,0
194,0,0,0,0
195,0,0,0,0
196,0,0,0,0


In [89]:
df_new['rf_1'] = (df_new['rf'] == df_new['top50'])
df_new['gb_1'] = (df_new['gb'] == df_new['top50'])
df_new['svm_1'] = (df_new['svm'] == df_new['top50'])

df_new['rf_1'] = df_new['rf_1'].astype(int)
df_new['gb_1'] = df_new['gb_1'].astype(int)
df_new['svm_1'] = df_new['svm_1'].astype(int)

df_new

,rf,gb,svm,top50,rf_1,gb_1,svm_1
0,0,0,0,0,1,1,1
1,1,1,1,1,1,1,1
2,0,0,0,0,1,1,1
3,0,0,0,0,1,1,1
4,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...
193,0,0,0,0,1,1,1
194,0,0,0,0,1,1,1
195,0,0,0,0,1,1,1
196,0,0,0,0,1,1,1


In [90]:
df_new["svm_1"].value_counts()

1    183
0     15
Name: svm_1, dtype: int64

In [91]:
df_new1 = df_new.drop(["rf","gb","svm","top50"], axis=1)

In [92]:
df_new1

,rf_1,gb_1,svm_1
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
...,...,...,...
193,1,1,1
194,1,1,1
195,1,1,1
196,1,1,1


#DEFAULT

In [94]:
X_train_de = X_train.drop(['bert_nega','bert_posi','bert_score','nltk_neg',
                        'nltk_neu','nltk_pos', 'nltk_score'], axis=1)

X_test_de = X_test.drop(['bert_nega','bert_posi','bert_score','nltk_neg',
                        'nltk_neu','nltk_pos', 'nltk_score'], axis=1)

In [95]:
# RF

forest = RandomForestClassifier(n_estimators=130, max_features=2, n_jobs=-1, random_state=0)
forest.fit(X_train_de, y_train)

y_pred = forest.predict(X_test_de)

train_accu_rf = forest.score(X_train_de, y_train)
print("Train model accuracy:{}".format(train_accu_rf))

test_acc_rf = accuracy_score(y_test, y_pred)
print("Test accuracy:{}".format(test_acc_rf))

confusion = confusion_matrix(y_test, y_pred)
print(confusion)

<ipython-input-95-98e038efdd00>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X_train_de, y_train)


Train model accuracy:1.0
Test accuracy:0.9444444444444444
[[145   5]
 [  6  42]]


In [96]:
y_pred_rf_de = y_pred

In [97]:
# GB

gb = GradientBoostingClassifier(learning_rate=0.05, n_estimators=60, max_depth=3)
gb.fit(X_train_de, y_train)
y_pred_gb = gb.predict(X_test_de)
confusion_gb = confusion_matrix(y_test, y_pred_gb)
print(confusion_gb)

train_accu_gb = gb.score(X_train_de, y_train)
print("Train model accuracy:{}".format(train_accu_gb))

test_acc_gb = accuracy_score(y_test, y_pred_gb)
print("Test accuracy:{}".format(test_acc_gb))

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[147   3]
 [  3  45]]
Train model accuracy:1.0
Test accuracy:0.9696969696969697


In [98]:
y_pred_gb_de = y_pred_gb

In [99]:
# SVM with MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train_de)
X_train_scaled = scaler.transform(X_train_de)
X_test_scaled = scaler.transform(X_test_de)

svc = SVC(kernel="rbf", C=10, gamma=0.01)
svc.fit(X_train_scaled, y_train)

y_pred_svc = svc.predict(X_test_scaled)
confusion_svc = confusion_matrix(y_test, y_pred_svc)
print(confusion_svc)

train_accu_svc = svc.score(X_train_scaled, y_train)
print("Train model accuracy:{}".format(train_accu_svc))

test_acc_svc = accuracy_score(y_test, y_pred_svc)
print("Train model accuracy:{}".format(test_acc_svc))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[145   5]
 [  9  39]]
Train model accuracy:0.961082910321489
Train model accuracy:0.9292929292929293


In [100]:
y_pred_svc_de = y_pred_svc

In [101]:
df1 = pd.DataFrame(y_pred_rf_de, columns=["rf"])
df2 = pd.DataFrame(y_pred_gb_de, columns=["gb"])
df3 = pd.DataFrame(y_pred_svc_de, columns=["svm"])

df_de = pd.concat([df1, df2, df3, y_test], axis=1)
df_de

,rf,gb,svm,top50
0,0,0,0,0
1,1,1,1,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
193,0,0,0,0
194,0,0,0,0
195,0,0,0,0
196,0,0,0,0


In [102]:
df_de['rf_1'] = (df_de['rf'] == df_de['top50'])
df_de['gb_1'] = (df_de['gb'] == df_de['top50'])
df_de['svm_1'] = (df_de['svm'] == df_de['top50'])

df_de['rf_1'] = df_de['rf_1'].astype(int)
df_de['gb_1'] = df_de['gb_1'].astype(int)
df_de['svm_1'] = df_de['svm_1'].astype(int)

df_de

,rf,gb,svm,top50,rf_1,gb_1,svm_1
0,0,0,0,0,1,1,1
1,1,1,1,1,1,1,1
2,0,0,0,0,1,1,1
3,0,0,0,0,1,1,1
4,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...
193,0,0,0,0,1,1,1
194,0,0,0,0,1,1,1
195,0,0,0,0,1,1,1
196,0,0,0,0,1,1,1


In [103]:
df_de1 = df_de.drop(["rf","gb","svm","top50"], axis=1)

#RESULT

In [104]:
# RF

tn = 145
fp = 5
fn = 6
tp = 42

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------New---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))


tn = 145
fp = 5
fn = 6
tp = 42

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------De---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

-----------------New---------------------
Accu: 0.9444444444444444
Prec: 0.8936170212765957
Recall: 0.875
F-measure: 0.892886345053148
-----------------De---------------------
Accu: 0.9444444444444444
Prec: 0.8936170212765957
Recall: 0.875
F-measure: 0.892886345053148


In [105]:
# GB

tn = 148
fp = 2
fn = 3
tp = 45

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------New---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))


tn = 147
fp = 3
fn = 3
tp = 45

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------De---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

-----------------New---------------------
Accu: 0.9747474747474747
Prec: 0.9574468085106383
Recall: 0.9375
F-measure: 0.9566639411283729
-----------------De---------------------
Accu: 0.9696969696969697
Prec: 0.9375
Recall: 0.9375
F-measure: 0.9375000000000001


In [106]:
# SVM

tn = 145
fp = 5
fn = 10
tp = 38

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------New---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))


tn = 145
fp = 5
fn = 9
tp = 39

accu = (tp+tn)/(tp+tn+fp+fn)
prec = tp/(tp+fp)
recall = tp/(tp+fn)
f_m = (1.04*prec*recall)/(0.04*prec+recall)

print('-----------------De---------------------')
print('Accu: {}'.format(accu))
print('Prec: {}'.format(prec))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(f_m))

-----------------New---------------------
Accu: 0.9242424242424242
Prec: 0.8837209302325582
Recall: 0.7916666666666666
F-measure: 0.8797862867319679
-----------------De---------------------
Accu: 0.9292929292929293
Prec: 0.8863636363636364
Recall: 0.8125
F-measure: 0.8832752613240419
